## 一、GPT Store：创建自己的 GPT

<img src="create_gpt.png" width=800px />

<img src="gpt.png" width=800px />

发布链接：https://chat.openai.com/g/g-iU8hVr4jR-wo-de-demogpt


## 二、Assistants API


In [1]:
# !pip install --upgrade openai

### 2.1、创建一个 Assistant

可以为每个应用，甚至应用中的每个有对话历史的使用场景，创建一个 assistant。

虽然可以用代码创建，也不复杂，例如：

```python
from openai import OpenAI

# 初始化 OpenAI 服务
client = OpenAI()

# 创建助手
assistant = client.beta.assistants.create(
    name="AGIClass Demo",
    instructions="你叫瓜瓜，你是AGI课堂的智能助理。你负责回答与AGI课堂有关的问题。",
    model="gpt-4-turbo",
)
```

但是，更佳做法是，到 [Playground](https://platform.openai.com/playground?mode=assistant) 在线创建，因为：

1. 更方便调整
2. 更方便测试


In [4]:
# 初始化
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import json

_ = load_dotenv(find_dotenv())

client = OpenAI()

# 创建助手
assistant = client.beta.assistants.create(
    name="AGIClass Demo2",
    instructions="你叫瓜瓜，你是AGI课堂的智能助理。你负责回答与AGI课堂有关的问题。",
    model="gpt-4-turbo",
)

print(assistant.id)

asst_5AZ7kfmj28ImuckNYHUAffM2


### 2.2、样例 Assistant 的配置

Instructions:

```
你叫瓜瓜。你是AGI课堂的助手。你只回答跟AI大模型有关的问题。不要跟学生闲聊。每次回答问题前，你要拆解问题并输出一步一步的思考过程。
```

Functions:

```JSON
{
  "name": "ask_database",
  "description": "Use this function to answer user questions about course schedule. Output should be a fully formed SQL query.",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {
        "type": "string",
        "description": "SQL query extracting info to answer the user's question.\nSQL should be written using this database schema:\n\nCREATE TABLE Courses (\n\tid INT AUTO_INCREMENT PRIMARY KEY,\n\tcourse_date DATE NOT NULL,\n\tstart_time TIME NOT NULL,\n\tend_time TIME NOT NULL,\n\tcourse_name VARCHAR(255) NOT NULL,\n\tinstructor VARCHAR(255) NOT NULL\n);\n\nThe query should be returned in plain text, not in JSON.\nThe query should only contain grammars supported by SQLite."
      }
    },
    "required": [
      "query"
    ]
  }
}
```


## 三、代码访问 Assistant

### 3.1、管理 thread

Threads：

1. Threads 里保存的是对话历史，即 messages
2. 一个 assistant 可以有多个 thread
3. 一个 thread 可以有无限条 message
4. 一个用户与 assistant 的多轮对话历史可以维护在一个 thread 里


In [5]:
import json


def show_json(obj):
    """把任意对象用排版美观的 JSON 格式打印出来"""
    print(json.dumps(
        json.loads(obj.model_dump_json()),
        indent=4,
        ensure_ascii=False
    ))

In [6]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# 初始化 OpenAI 服务
client = OpenAI()   # openai >= 1.3.0 起，OPENAI_API_KEY 和 OPENAI_BASE_URL 会被默认使用

# 创建 thread
thread = client.beta.threads.create()
show_json(thread)

{
    "id": "thread_WmDT6y5JYHbyMHHzK7bKUYcO",
    "created_at": 1715686479,
    "metadata": [],
    "object": "thread",
    "tool_resources": []
}


In [7]:
thread = client.beta.threads.create(
    metadata={"fullname": "王卓然", "username": "taliux"}
)
show_json(thread)

{
    "id": "thread_wTiNixiUIiumkovo2NKjjduE",
    "created_at": 1715686522,
    "metadata": {
        "fullname": "王卓然",
        "username": "taliux"
    },
    "object": "thread",
    "tool_resources": []
}


In [8]:
thread = client.beta.threads.retrieve(thread.id)
show_json(thread)

{
    "id": "thread_wTiNixiUIiumkovo2NKjjduE",
    "created_at": 1715686522,
    "metadata": {
        "fullname": "王卓然",
        "username": "taliux"
    },
    "object": "thread",
    "tool_resources": {
        "code_interpreter": {
            "file_ids": []
        }
    }
}


此外，还有：

1. `threads.modify()` 修改 thread 的 `metadata` 和 `tool_resources`
2. `threads.retrieve()` 获取 thread
3. `threads.delete()` 删除 thread。

具体文档参考：https://platform.openai.com/docs/api-reference/threads


### 3.2、给 Threads 添加 Messages

这里的 messages 结构要复杂一些：

1.  不仅有文本，还可以有图片和文件
2.  也有 `metadata`


In [9]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,  # message 必须归属于一个 thread
    role="user",          # 取值是 user 或者 assistant。但 assistant 消息会被自动加入，我们一般不需要自己构造
    content="你都能做什么？",
)
show_json(message)

{
    "id": "msg_O9YiJE2ofv86V1BNfwWZ08Ka",
    "assistant_id": null,
    "content": [
        {
            "text": {
                "annotations": [],
                "value": "你都能做什么？"
            },
            "type": "text"
        }
    ],
    "created_at": 1715686573,
    "file_ids": null,
    "metadata": [],
    "object": "thread.message",
    "role": "user",
    "run_id": null,
    "thread_id": "thread_wTiNixiUIiumkovo2NKjjduE",
    "attachments": []
}


还有如下函数：

1. `threads.messages.retrieve()` 获取 message
2. `threads.messages.update()` 更新 message 的 `metadata`
3. `threads.messages.list()` 列出给定 thread 下的所有 messages

具体文档参考：https://platform.openai.com/docs/api-reference/messages

也可以在创建 thread 同时初始化一个 message 列表


In [10]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "你好",
        },
        {
            "role": "assistant",
            "content": "有什么可以帮您？",
        },
        {
            "role": "user",
            "content": "你是谁？",
        },
    ]
)

show_json(thread)  # 显示 thread
print("-----")
show_json(client.beta.threads.messages.list(
    thread.id))  # 显示指定 thread 中的 message 列表

{
    "id": "thread_9oS5kfgX2FbWGZxomPbMQk6S",
    "created_at": 1715686620,
    "metadata": [],
    "object": "thread",
    "tool_resources": []
}
-----
{
    "data": [
        {
            "id": "msg_7eo6I9bOr5X4uNQgrI4q3IX2",
            "assistant_id": null,
            "content": [
                {
                    "text": {
                        "annotations": [],
                        "value": "你是谁？"
                    },
                    "type": "text"
                }
            ],
            "created_at": 1715686620,
            "file_ids": null,
            "metadata": [],
            "object": "thread.message",
            "role": "user",
            "run_id": null,
            "thread_id": "thread_9oS5kfgX2FbWGZxomPbMQk6S",
            "attachments": []
        },
        {
            "id": "msg_UOkXgw8XKhw8sNp5rANnYLWy",
            "assistant_id": null,
            "content": [
                {
                    "text": {
                        "anno

## 3.3、开始 Run

- 用 run 把 assistant 和 thread 关联，进行对话
- 一个 prompt 就是一次 run

### 3.1、直接运行


In [12]:
assistant_id = "asst_5AZ7kfmj28ImuckNYHUAffM2"  # 从 Playground 中拷贝

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

AttributeError: 'Runs' object has no attribute 'create_and_poll'

In [13]:
if run.status == 'completed':
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    show_json(messages)
else:
    print(run.status)

NameError: name 'run' is not defined

### 3.2、Run 的状态（选）

Run 的底层是个异步调用，意味着它不等大模型处理完，就返回。我们通过 `run.status` 了解大模型的工作进展情况，来判断下一步该干什么。

`run.status` 有的状态，和状态之间的转移关系如图。

<img src="statuses.png" width="800" />



### 3.3、流式运行

1. 创建回调函数


In [14]:
from typing_extensions import override
from openai import AssistantEventHandler


class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        """响应输出创建事件"""
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        """响应输出生成的流片段"""
        print(delta.value, end="", flush=True)

ImportError: cannot import name 'AssistantEventHandler' from 'openai' (/root/miniconda3/envs/ai_zhihu/lib/python3.10/site-packages/openai/__init__.py)

2. 运行 run


In [15]:
# 添加新一轮的 user message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="你说什么？",
)
# 使用 stream 接口并传入 EventHandler
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant_id,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

AttributeError: 'Runs' object has no attribute 'stream'

还有如下函数：

1. `threads.runs.list()` 列出 thread 归属的 run
2. `threads.runs.retrieve()` 获取 run
3. `threads.runs.update()` 修改 run 的 metadata
4. `threads.runs.cancel()` 取消 `in_progress` 状态的 run

具体文档参考：https://platform.openai.com/docs/api-reference/runs


<div class="alert alert-warning">
<b>思考：</b> 进一步理解 run 与 thread 的设计
<ul>
    <li>抛开 Assistants API，假设你要开发任意一个多轮对话的 AI 机器人</li>
    <li>从架构设计的角度，应该怎么维护用户、对话历史、对话引擎、对话服务？</li>
</ul>
</div>


## 四、使用 Tools

### 4.1、创建 Assistant 时声明 Code_Interpreter


如果用代码创建：

```python
assistant = client.beta.assistants.create(
    name="Demo Assistant",
    instructions="你是人工智能助手。你可以通过代码回答很多数学问题。",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-turbo"
)
```

在回调中加入 code_interpreter 的事件响应


In [16]:
from typing_extensions import override
from openai import AssistantEventHandler


class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        """响应输出创建事件"""
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        """响应输出生成的流片段"""
        print(delta.value, end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        """响应工具调用"""
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_tool_call_delta(self, delta, snapshot):
        """响应工具调用的流片段"""
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
        if delta.code_interpreter.outputs:
            print(f"\n\noutput >", flush=True)
            for output in delta.code_interpreter.outputs:
                if output.type == "logs":
                    print(f"\n{output.logs}", flush=True)

ImportError: cannot import name 'AssistantEventHandler' from 'openai' (/root/miniconda3/envs/ai_zhihu/lib/python3.10/site-packages/openai/__init__.py)